In [275]:
import numpy as np
import pandas as pd
import pickle
import re

In [2]:
fields = pd.read_csv("../data/raw/fields.csv")

In [7]:
df2011 = pd.read_csv("../data/raw/all2011.csv", low_memory=False, header=None)

In [8]:
fields.head()

,Field number,Description,Header
0,0,Game ID,GAME_ID
1,1,Visiting team,AWAY_TEAM_ID
2,2,Inning,INN_CT
3,3,Batting team,BAT_HOME_ID
4,4,Outs,OUTS_CT


In [10]:
df2011.columns = fields.Header

In [12]:
df2011.head()

Header,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,PITCH_SEQ_TX,AWAY_SCORE_CT,HOME_SCORE_CT,...,REMOVED_FOR_PH_BAT_FLD_CD,PO1_FLD_CD,PO2_FLD_CD,PO3_FLD_CD,ASS1_FLD_CD,ASS2_FLD_CD,ASS3_FLD_CD,ASS4_FLD_CD,ASS5_FLD_CD,EVENT_ID
0,ANA201104080,TOR,1,0,0,1,2,FBSX,0,0,...,0,9,0,0,0,0,0,0,0,1
1,ANA201104080,TOR,1,0,1,0,0,X,0,0,...,0,9,0,0,0,0,0,0,0,2
2,ANA201104080,TOR,1,0,2,1,2,CBCS,0,0,...,0,2,0,0,0,0,0,0,0,3
3,ANA201104080,TOR,1,1,0,3,1,CBBBB,0,0,...,0,0,0,0,0,0,0,0,0,4
4,ANA201104080,TOR,1,1,0,2,2,BCSBS,0,0,...,0,2,0,0,0,0,0,0,0,5


In [22]:
df_plays = df2011[['GAME_ID', 'EVENT_ID', 'AWAY_TEAM_ID', 'INN_CT', 'BAT_HOME_ID', 
                   'OUTS_CT', 'EVENT_OUTS_CT', 'RBI_CT',
                   'BASE1_RUN_ID', 'BASE2_RUN_ID', 'BASE3_RUN_ID',
                   'BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 'RUN3_DEST_ID']]

In [24]:
df_plays.head(20)

Header,GAME_ID,EVENT_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,EVENT_OUTS_CT,RBI_CT,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID
0,ANA201104080,1,TOR,1,0,0,1,0,NaN,NaN,NaN,0,0,0,0
1,ANA201104080,2,TOR,1,0,1,1,0,NaN,NaN,NaN,0,0,0,0
2,ANA201104080,3,TOR,1,0,2,1,0,NaN,NaN,NaN,0,0,0,0
3,ANA201104080,4,TOR,1,1,0,0,0,NaN,NaN,NaN,1,0,0,0
4,ANA201104080,5,TOR,1,1,0,1,0,iztum001,NaN,NaN,0,1,0,0
5,ANA201104080,6,TOR,1,1,1,1,0,iztum001,NaN,NaN,0,0,0,0
6,ANA201104080,7,TOR,1,1,2,0,0,NaN,NaN,NaN,1,0,0,0
7,ANA201104080,8,TOR,1,1,2,0,0,abreb001,NaN,NaN,3,5,0,0
8,ANA201104080,9,TOR,1,1,2,1,0,NaN,NaN,huntt001,0,0,0,3
9,ANA201104080,10,TOR,2,0,0,1,0,NaN,NaN,NaN,0,0,0,0


In [44]:
df_plays['initial_runner_on_first_string'] = df_plays.BASE1_RUN_ID.map(lambda x: '' if x is np.nan else '1')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
df_plays['initial_runner_on_first_string'].head()

0     
1     
2     
3     
4    1
Name: initial_runner_on_first_string, dtype: object

In [46]:
df_plays['initial_runner_on_second_string'] = df_plays.BASE2_RUN_ID.map(lambda x: '' if x is np.nan else '2')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
df_plays['initial_runner_on_third_string'] = df_plays.BASE3_RUN_ID.map(lambda x:'' if x is np.nan else '3')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
df_plays['initial_bases_string_123'] = (df_plays['initial_runner_on_first_string'] + 
                                        df_plays['initial_runner_on_second_string'] +
                                        df_plays['initial_runner_on_third_string'])

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
df_plays['initial_bases_string'] = df_plays['initial_bases_string_123'].map(lambda x: x if x else '0')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
df_plays.groupby('initial_bases_string')['initial_bases_string'].value_counts()

initial_bases_string  initial_bases_string
0                     0                       103853
1                     1                        37559
12                    12                       13085
123                   123                       4408
13                    13                        6092
2                     2                        17274
23                    23                        4133
3                     3                         5460
Name: initial_bases_string, dtype: int64

In [67]:
df_plays['final_runner_on_first_flag'] = (df_plays['BAT_DEST_ID'] == 1) | (df_plays['RUN1_DEST_ID'] == 1)

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [69]:
df_plays['final_runner_on_second_flag'] = ((df_plays['BAT_DEST_ID'] == 2) 
                                           | (df_plays['RUN1_DEST_ID'] == 2) 
                                           | (df_plays['RUN2_DEST_ID'] == 2))

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
df_plays['final_runner_on_third_flag'] = ((df_plays['BAT_DEST_ID'] == 3) 
                                    | (df_plays['RUN1_DEST_ID'] == 3) 
                                    | (df_plays['RUN2_DEST_ID'] == 3)
                                    | (df_plays['RUN3_DEST_ID'] == 3))

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [75]:
df_plays['final_runner_on_first_string'] = df_plays.final_runner_on_first_flag.map(lambda x: 
                                                              '' if x is False else '1')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
df_plays['final_runner_on_first_string'].head(10)

0     
1     
2     
3    1
4    1
5     
6    1
7     
8     
9     
Name: final_runner_on_first_string, dtype: object

In [78]:
df_plays['final_runner_on_second_string'] = df_plays.final_runner_on_second_flag.map(lambda x: 
                                                              '' if x is False else '2')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
df_plays['final_runner_on_third_string'] = df_plays.final_runner_on_third_flag.map(lambda x: 
                                                              '' if x is False else '3')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
df_plays['final_bases_string_123'] = (df_plays['final_runner_on_first_string'] + 
                                        df_plays['final_runner_on_second_string'] +
                                        df_plays['final_runner_on_third_string'])

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
df_plays['final_bases_string'] = df_plays['final_bases_string_123'].map(lambda x: x if x else '0')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
df_plays.groupby('final_bases_string')['final_bases_string'].value_counts()

final_bases_string  final_bases_string
0                   0                     79955
1                   1                     45561
12                  12                    17115
123                 123                    5829
13                  13                     7848
2                   2                     22443
23                  23                     5461
3                   3                      7652
Name: final_bases_string, dtype: int64

In [103]:
df_plays.OUTS_CT.head()

0    0
1    1
2    2
3    0
4    0
Name: OUTS_CT, dtype: int64

In [104]:
df_plays['initial_outs_string'] = df_plays.OUTS_CT.map(lambda x: '' if x==0 else 'X' if x==1 else 'XX')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [105]:
df_plays['final_outs_ct'] = df_plays.OUTS_CT + df_plays.EVENT_OUTS_CT

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
df_plays['final_outs_string'] = df_plays.final_outs_ct.map(
          lambda x: '' if x==0 else 'X' if x==1 else 'XX' if x==2 else 'XXX')

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [107]:
df_plays['final_outs_string'].head(10)

0      X
1     XX
2    XXX
3       
4      X
5     XX
6     XX
7     XX
8    XXX
9      X
Name: final_outs_string, dtype: object

In [108]:
df_plays['trans3'] = (df_plays.initial_bases_string
                      + df_plays.initial_outs_string 
                      + ':' 
                      + df_plays.final_bases_string
                      + df_plays.final_outs_string)

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [109]:
df_plays['trans3'].head(15)

0         0:0X
1       0X:0XX
2     0XX:0XXX
3          0:1
4         1:1X
5       1X:0XX
6      0XX:1XX
7      1XX:3XX
8     3XX:3XXX
9         0:0X
10      0X:0XX
11    0XX:0XXX
12         0:1
13       1:0XX
14    0XX:0XXX
Name: trans3, dtype: object

In [198]:
transition_counts = pd.DataFrame(df_plays.groupby(['trans3']).size())

In [199]:
transition_counts.head()

,0
trans3,
0:0,1239
0:0X,30710
0:1,10866
0:2,2266
0:3,262


In [200]:
transition_counts.index

Index(['0:0', '0:0X', '0:1', '0:2', '0:3', '0X:0X', '0X:0XX', '0X:1X', '0X:2X',
       '0X:3X',
       ...
       '3X:3X', '3X:3XX', '3XX:0XX', '3XX:0XXX', '3XX:13XX', '3XX:1XX',
       '3XX:1XXX', '3XX:2XX', '3XX:3XX', '3XX:3XXX'],
      dtype='object', name='trans3', length=313)

In [201]:
transition_counts.columns = ['counts']

In [202]:
transition_counts.head()

,counts
trans3,
0:0,1239
0:0X,30710
0:1,10866
0:2,2266
0:3,262


In [203]:
transition_counts.reset_index(inplace=True)

In [204]:
transition_counts['from'] = transition_counts.trans3.map(lambda x: re.search('^[0-3]+X*:',x).group(0))

In [205]:
transition_counts.head(10)

,trans3,counts,from
0,0:0,1239,0:
1,0:0X,30710,0:
2,0:1,10866,0:
3,0:2,2266,0:
4,0:3,262,0:
5,0X:0X,819,0X:
6,0X:0XX,22297,0X:
7,0X:1X,7889,0X:
8,0X:2X,1537,0X:
9,0X:3X,152,0X:


In [206]:
from_counts = pd.DataFrame(transition_counts.groupby('from').sum())

In [207]:
from_counts.head()

,counts
from,
0:,45343
0X:,32694
0XX:,25816
123:,667
123X:,1694


In [208]:
from_count_series = from_counts.loc[transition_counts['from'],'counts']

In [209]:
transition_counts['from_count'] = list(from_count_series)

In [210]:
transition_counts['probs'] = transition_counts['counts']/transition_counts['from_count']

In [211]:
transition_counts.head(20)

,trans3,counts,from,from_count,probs
0,0:0,1239,0:,45343,0.027325
1,0:0X,30710,0:,45343,0.677282
2,0:1,10866,0:,45343,0.239640
3,0:2,2266,0:,45343,0.049975
4,0:3,262,0:,45343,0.005778
5,0X:0X,819,0X:,32694,0.025050
6,0X:0XX,22297,0X:,32694,0.681991
7,0X:1X,7889,0X:,32694,0.241298
8,0X:2X,1537,0X:,32694,0.047012
9,0X:3X,152,0X:,32694,0.004649


In [212]:
df_plays['home_team'] = df_plays['GAME_ID'].map(lambda x: x[0:3])

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [220]:
team_transition_counts = pd.DataFrame(df_plays.groupby(['home_team','trans3']).size())

In [221]:
team_transition_counts.head(20)

0
home_team trans3        
ANA       0:0         34
          0:0X      1052
          0:1        363
          0:2         60
          0:3          7
          0X:0X       30
          0X:0XX     766
          0X:1X      267
          0X:2X       52
          0X:3X        5
          0XX:0XX     20
          0XX:0XXX   596
          0XX:1XX    208
          0XX:2XX     47
          0XX:3XX      6
          123:12       1
          123:123      4
          123:123X     5
          123:12X      2
          123:13X      1

In [222]:
team_transition_counts.columns = ['counts']

In [225]:
team_transition_counts.reset_index(inplace=True)

In [226]:
team_transition_counts['from'] = team_transition_counts.trans3.map(lambda x: re.search('^[0-3]+X*:',x).group(0))

In [227]:
team_transition_counts.head()

,home_team,trans3,counts,from
0,ANA,0:0,34,0:
1,ANA,0:0X,1052,0:
2,ANA,0:1,363,0:
3,ANA,0:2,60,0:
4,ANA,0:3,7,0:


In [228]:
team_from_counts = pd.DataFrame(team_transition_counts.groupby(['home_team', 'from']).sum())

In [229]:
team_from_counts.head()

counts
home_team from         
ANA       0:       1516
          0X:      1120
          0XX:      877
          123:       16
          123X:      41

In [246]:
team_from_counts.reset_index(inplace=True)

In [248]:
team_transition_counts.head()

,home_team,trans3,counts,from
0,ANA,0:0,34,0:
1,ANA,0:0X,1052,0:
2,ANA,0:1,363,0:
3,ANA,0:2,60,0:
4,ANA,0:3,7,0:


In [249]:
team_from_counts.head()

,home_team,from,counts
0,ANA,0:,1516
1,ANA,0X:,1120
2,ANA,0XX:,877
3,ANA,123:,16
4,ANA,123X:,41


In [250]:
team_transition_counts['from_team'] = team_transition_counts['from'] + team_transition_counts['home_team']

In [251]:
team_transition_counts.head()

,home_team,trans3,counts,from,from_team
0,ANA,0:0,34,0:,0:ANA
1,ANA,0:0X,1052,0:,0:ANA
2,ANA,0:1,363,0:,0:ANA
3,ANA,0:2,60,0:,0:ANA
4,ANA,0:3,7,0:,0:ANA


In [252]:
team_from_counts['from_team'] = team_from_counts['from'] + team_from_counts['home_team']

In [254]:
team_from_counts.head()

,home_team,from,counts,from_team
0,ANA,0:,1516,0:ANA
1,ANA,0X:,1120,0X:ANA
2,ANA,0XX:,877,0XX:ANA
3,ANA,123:,16,123:ANA
4,ANA,123X:,41,123X:ANA


In [257]:
team_from_counts.set_index('from_team', inplace=True)

In [259]:
team_transition_counts.set_index('from_team', inplace=True)

In [267]:
joined_team_counts = team_transition_counts.join(team_from_counts,how='left',rsuffix='frm')

In [268]:
joined_team_counts.head()

,home_team,trans3,counts,from,home_teamfrm,fromfrm,countsfrm
from_team,,,,,,,
0:ANA,ANA,0:0,34,0:,ANA,0:,1516
0:ANA,ANA,0:0X,1052,0:,ANA,0:,1516
0:ANA,ANA,0:1,363,0:,ANA,0:,1516
0:ANA,ANA,0:2,60,0:,ANA,0:,1516
0:ANA,ANA,0:3,7,0:,ANA,0:,1516


In [269]:
joined_team_counts['probs'] = joined_team_counts['counts']/joined_team_counts['countsfrm']

In [270]:
joined_team_counts.head()

,home_team,trans3,counts,from,home_teamfrm,fromfrm,countsfrm,probs
from_team,,,,,,,,
0:ANA,ANA,0:0,34,0:,ANA,0:,1516,0.022427
0:ANA,ANA,0:0X,1052,0:,ANA,0:,1516,0.693931
0:ANA,ANA,0:1,363,0:,ANA,0:,1516,0.239446
0:ANA,ANA,0:2,60,0:,ANA,0:,1516,0.039578
0:ANA,ANA,0:3,7,0:,ANA,0:,1516,0.004617


In [272]:
data2011 = joined_team_counts.pivot(index='home_team', columns='trans3', values='probs')

In [274]:
data2011.fillna(0, inplace=True)

In [276]:
f = open("data2011.pkl","wb")

In [277]:
pickle.dump(data2011,f)

In [278]:
f.close()